### This notebook contains the solution of Data Science Capstone Week 3 Assignment

Web scraping has been done using Python **BeautifulSoup** library

In [1]:

# insall beautifulsoup library to scrape the web page
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /home/abhiroop43/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> anaconda::conda-4.8.3-py37_0


Preparing transaction: / done
Verifying transaction: \ | done
Executing transaction: - \ done


Add the necessary imports


In [2]:
import bs4 as bs
import urllib.request
import pandas as pd

Scrape the provided web page to get the details of the boroughs

In [3]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')

Parse the html rows obtained by scraping the web page and add them to a dataframe

In [4]:
data_rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    data_rows.append(row)
df = pd.DataFrame(data_rows, columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


Perform some data cleanup on the generated dataframe

In [5]:
df.dropna(axis=0, inplace=True)
df.replace('\n','', regex=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove rows with no assigned boroughs

In [6]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Assign Borough names to Neighborhoods where neighborhood is not assigned

In [7]:
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)

df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape # print the shape of the dataframe

(103, 3)

In [24]:
# get the coordinates from csv

df_cood = pd.read_csv('Geospatial_Coordinates.csv')
df_cood.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_cood.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine the dataframes using PostalCode column

In [25]:
df_merged = pd.merge(df, df_cood, on='PostalCode')
df_merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
